<a href="https://colab.research.google.com/github/kevin-rego/docstring-generator/blob/main/Python_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ast
import pandas as pd
import requests

# Replace these variables as per your use case
owner = 'pandas-dev'
repo = 'pandas'
token = '' # Insert Github PAT token


# Create a DataFrame
df = pd.DataFrame(columns=['File', 'Function', 'Docstring'])

# API URL for repository content
url = f'https://api.github.com/repos/{owner}/{repo}/git/trees/main?recursive=1'
headers = {'Authorization': f'token {token}'}

# API Request for repository content
response = requests.get(url, headers=headers)
tree = response.json()

# Check if request was successful
if response.status_code == 200:
    # Loop through all files in the repository
    for item in tree['tree']:
        # Check if item is a Python file
        if item['path'].endswith('.py'):
            # API request to get file content
            file_url = f'https://api.github.com/repos/{owner}/{repo}/contents/{item["path"]}'
            file_response = requests.get(file_url, headers=headers)

            # Check if request for file content was successful
            if file_response.status_code == 200:
                file_content = requests.get(file_response.json()['download_url']).text

                # Parsing file content with AST
                tree = ast.parse(file_content)
                for node in tree.body:
                    if isinstance(node, ast.FunctionDef):
                        # Check for docstring
                        docstring = ast.get_docstring(node)
                        if docstring:
                            # Strip docstring from function code
                            node.body = [n for n in node.body if not isinstance(n, ast.Expr) or not isinstance(n.value, ast.Str)]
                            function_code = ast.unparse(node)  # Available in Python 3.9+

                            # Append to DataFrame
                            df = df.append({
                                'File': item['path'],
                                'Function': function_code,
                                'Docstring': f'"""{docstring}"""'
                            }, ignore_index=True)
else:
    print("Failed to retrieve repository content")

# Export the DataFrame to csv
df.to_csv('output.csv', index=False)


<ipython-input-9-97f419eaa8ff>:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
<ipython-input-9-97f419eaa8ff>:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
<ipython-input-9-97f419eaa8ff>:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
<ipython-input-9-97f419eaa8ff>:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
<ipython-input-9-97f419eaa8ff>:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
<ipython-input-9-97f419eaa8ff>:48: FutureWarning: The frame.appen

KeyboardInterrupt: ignored

In [ ]:
df.to_csv('output.csv', index=False)